In [29]:
import pandas as pd
import numpy as np
from IPython.display import display

### <font color="orange">データの読み込み</font>

In [30]:
import zipfile
import os
# 解凍したいzipファイルのパス
zip_file_path = '../../../../0_original_datasets/datasets.zip'

# 解凍先のディレクトリ
folder_path = '../../../tmp_data_folder/0_tmp_original_data/'

if os.path.exists(folder_path):
    print("既に解凍されたフォルダが存在します。")
    pass
else:
    # Zipファイルを開いて解凍する
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)

    print(f'{zip_file_path} を {folder_path} に解凍しました。')

../../../../0_original_datasets/datasets.zip を ../../../tmp_data_folder/0_tmp_original_data/ に解凍しました。


In [31]:
olist_order_payments_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_order_payments_dataset.csv")
olist_order_items_dataset = pd.read_csv("../../../tmp_data_folder/0_tmp_original_data/datasets/olist_order_items_dataset.csv")


# payment_type=="not_defined"は支払いが無いので、行を削除する。
display(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"])
olist_order_payments_dataset = olist_order_payments_dataset.drop(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"].index)
display(olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="not_defined"])

,order_id,payment_sequential,payment_type,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.0
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.0


,order_id,payment_sequential,payment_type,payment_installments,payment_value


### <font color="orange">データの確認</font>

In [32]:
print(olist_order_payments_dataset[olist_order_payments_dataset.duplicated(subset="order_id", keep=False)].shape[0])
print(olist_order_items_dataset[olist_order_items_dataset.duplicated(subset="order_id", keep=False)].shape[0])

#* ともに重複有り。

7407
23787


In [33]:
# "olist_order_payment_dataset"  と  "olist_order_item_dataset"の両方で重複する行のみを抽出する。

# olist_order_payment_datasetで重複するorder_idを抽出する。
payment_duplication_order_id = olist_order_payments_dataset[olist_order_payments_dataset.duplicated(subset="order_id", keep="first")].order_id.values

# olist_order_item_datasetで重複するorder_idを抽出する。
item_duplication_order_id = olist_order_items_dataset[olist_order_items_dataset.duplicated(subset="order_id", keep="first")].order_id.values

# payment_duplication_order_id とitem_duplication_order_idで一致するorder_idのみ抽出する。
final_order_id = set(payment_duplication_order_id) & set(item_duplication_order_id)
final_order_id = list(final_order_id)

# データを確認する。
id = final_order_id[0]
display(olist_order_payments_dataset.query("order_id == @id"))
display(olist_order_items_dataset.query("order_id == @id"))

,order_id,payment_sequential,payment_type,payment_installments,payment_value
84919,6e6242aabab7596add1baa594d4e7208,2,credit_card,4,45.24
91090,6e6242aabab7596add1baa594d4e7208,1,credit_card,10,105.26


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
48577,6e6242aabab7596add1baa594d4e7208,1,389d119b48cf3043d311335e499d9c6b,1f50f920176fa81dab994f9023523100,2018-03-29 14:27:41,53.9,21.35
48578,6e6242aabab7596add1baa594d4e7208,2,389d119b48cf3043d311335e499d9c6b,1f50f920176fa81dab994f9023523100,2018-03-29 14:27:41,53.9,21.35


In [34]:
# order_idごとの
# "olist_order_payments_dataset"の "payment_value"の合計と、
# "olist_order_items_dataset" の "price"と"freight_value"の合計が一致しているように見えたので確認する。

tmp_payment = olist_order_payments_dataset.groupby("order_id", as_index=False)["payment_value"].sum()
tmp_payment["payment_value"] = tmp_payment["payment_value"].round(2)    #浮動小数点の計算をするため。np.isclose()というものもある。

tmp_item = olist_order_items_dataset.groupby("order_id", as_index=False)[["price", "freight_value"]].sum()
tmp_item["total_item_value"] = tmp_item["price"] + tmp_item["freight_value"]
tmp_item["total_item_value"] = tmp_item["total_item_value"].round(2)    #浮動小数点の計算をするため。np.isclose()というものもある。
tmp_item = tmp_item[["order_id", "total_item_value"]]

check_df = pd.merge(tmp_payment, tmp_item, on="order_id", how="inner")

check_df["diff"] = check_df["payment_value"] - check_df["total_item_value"].round(2)

print(f"一致 　: {len(check_df[check_df['diff'] == 0.0].sort_values(by='diff', ascending=False))}件")
print(f"不一致 : {len(check_df[check_df['diff'] != 0.0].sort_values(by='diff', ascending=False))}件")

print("="*20 + " 不一致データ一覧 " + "="*20)
check_df[check_df["diff"] != 0.0].sort_values(by="diff", ascending=False)

#* 支払金額の不一致が576件あった。
#* 分割払いの回数が多いデータなどはpayment_valueの方が大きい傾向にあるようだが、詳細な要因は不明。

一致 　: 98089件
不一致 : 576件
==================== 不一致データ一覧 ====================


,order_id,payment_value,total_item_value,diff
79537,ce6d150fb29ada17d2082f4847107665,1586.47,1403.66,182.81
42515,6e5fe7366a2e1bfbf3257dba0af1267f,406.92,287.91,119.01
43434,70b742795bc441e94a44a084b6d9ce7a,578.82,466.93,111.89
58752,996c7e73600ad3723e8627ab7bef81e4,664.43,587.90,76.53
43437,70b7e94ea46d3e8b5bc12a50186edaf0,274.84,213.15,61.69
...,...,...,...,...
42292,6dcf0aeb8b1eb4021c26e1d0e9394979,318.97,333.92,-14.95
25102,4154bf1348caac78152fe76e3e9c4af8,150.27,165.26,-14.99
42504,6e57e23ecac1ae881286657694444267,333.91,350.41,-16.50
97605,fd33085945f15975375cd8ec85440511,212.82,234.62,-21.80


### <font color="orange">データ作成"payments_dataset"について,order_idが一意になるように整形する。</font>

In [35]:
# payments_datasetについて
id = final_order_id[0]
olist_order_payments_dataset.query("order_id == @id")

,order_id,payment_sequential,payment_type,payment_installments,payment_value
84919,6e6242aabab7596add1baa594d4e7208,2,credit_card,4,45.24
91090,6e6242aabab7596add1baa594d4e7208,1,credit_card,10,105.26


In [36]:
# payment_sequentialは複数の支払い方法を選択したときに立つシーケンス番号なので、行数をカウントすれば良い。

custom_payments_dataset = olist_order_payments_dataset[["order_id"]].drop_duplicates("order_id", inplace=False)
count_payment_sequential = olist_order_payments_dataset.groupby("order_id")["payment_sequential"].count().to_dict()
custom_payments_dataset["count_payment_sequential"] = custom_payments_dataset["order_id"].map(count_payment_sequential)

In [37]:
# payment_typeについては、payment_typeの種類の数だけ列を作成し、フラグを立てる。

payment_type = olist_order_payments_dataset["payment_type"].unique().tolist()

for type in payment_type:
    column_name = "payment_type_"+str(type)
    custom_payments_dataset[column_name]=0
    
    tmp_filterd_df = olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]==type]["order_id"]
    custom_payments_dataset.loc[custom_payments_dataset["order_id"].isin(tmp_filterd_df), column_name] = 1


In [38]:
# payment_installmentsについては支払い方法がcredit_cardのときのみ、分割回数を指定可能。
# order_idごとにcredit_card払いの分割回数の平均を算出する。

credic_card_df = olist_order_payments_dataset[olist_order_payments_dataset["payment_type"]=="credit_card"]
mean_credit_card_payment_installments = credic_card_df.groupby("order_id")["payment_installments"].mean().to_dict()
custom_payments_dataset["mean_credit_card_payment_installments"] = custom_payments_dataset["order_id"].map(mean_credit_card_payment_installments)
custom_payments_dataset["mean_credit_card_payment_installments"].fillna(0, inplace=True)
custom_payments_dataset["mean_credit_card_payment_installments"] = custom_payments_dataset["mean_credit_card_payment_installments"].astype(int)

In [39]:
# payment_valueはorder_idごとに合計し、代入する。
sum_payment_value = olist_order_payments_dataset.groupby("order_id")["payment_value"].sum().to_dict()
custom_payments_dataset["payment_value"] = custom_payments_dataset["order_id"].map(sum_payment_value)

In [40]:
custom_payments_dataset.head()

,order_id,count_payment_sequential,payment_type_credit_card,payment_type_boleto,payment_type_voucher,payment_type_debit_card,mean_credit_card_payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,1,0,0,0,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,1,0,0,0,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,1,0,0,0,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,1,0,0,0,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,1,0,0,0,2,128.45


### <font color="orange">データの保存</font>

In [41]:
import os

save_path = "../../../tmp_data_folder/1_custom_data/"

# フォルダが存在しない場合に作成
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f'フォルダ {save_path} を作成しました。')
else:
    print(f'フォルダ {save_path} は既に存在します。')
    
custom_payments_dataset.to_csv("../../../tmp_data_folder/1_custom_data/custom_payments_dataset.csv", index=False)
print("../tmp/にunique_olist_review_dataset.csvを保存しました。")

フォルダ ../../../tmp_data_folder/1_custom_data/ は既に存在します。
../tmp/にunique_olist_review_dataset.csvを保存しました。
